In [1]:
PATH='/home/kirana/Documents/phd/exp3_autoencoder'

In [2]:
DATAPATH='/home/kirana/Documents/final_dissertation_final/experiments/datasets/ml-latest-small'

In [3]:
import pandas as pd
import numpy as np
import os
import datetime
import sklearn
import torch

In [4]:
ls {DATAPATH}/*.csv

/home/kirana/Documents/final_dissertation_final/experiments/datasets/ml-latest-small/dftraining_knn.csv
/home/kirana/Documents/final_dissertation_final/experiments/datasets/ml-latest-small/dftrain_knn.csv
/home/kirana/Documents/final_dissertation_final/experiments/datasets/ml-latest-small/dftrainvalid_knn.csv
/home/kirana/Documents/final_dissertation_final/experiments/datasets/ml-latest-small/dfvalid_knn.csv
/home/kirana/Documents/final_dissertation_final/experiments/datasets/ml-latest-small/links.csv
/home/kirana/Documents/final_dissertation_final/experiments/datasets/ml-latest-small/movies.csv
/home/kirana/Documents/final_dissertation_final/experiments/datasets/ml-latest-small/ratings.csv
/home/kirana/Documents/final_dissertation_final/experiments/datasets/ml-latest-small/tags.csv


## Read Files

In [5]:
ratings=pd.read_csv(f'{DATAPATH}/ratings.csv')
print (ratings.head())
movies=pd.read_csv (f'{DATAPATH}/movies.csv')
print(movies.head())
links=pd.read_csv(f'{DATAPATH}/links.csv')
print (links.head())
tags=pd.read_csv(f'{DATAPATH}/tags.csv')
print (tags.head())

   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
   movieId  imdbId   tmdbId
0        1  114709    862.0
1        2  113497   8844.0
2        3  113228  15602.0
3        4  114885  31357.0
4        5  113041  11862.0
   userId

In [6]:
ratings['timestamp']=pd.to_datetime(ratings['timestamp'],unit='s')
ratings=pd.merge(ratings,movies,on='movieId',how='left')
ratings=pd.merge(ratings,links,on='movieId',how='left')
tags=tags.groupby(['userId','movieId'],as_index=False)['tag'].agg(lambda x: ' '.join(x.values.astype(str)))
ratings=pd.merge(ratings, tags,on=['userId','movieId'],how='left')
print(ratings.shape)

(100836, 9)


## Random Cross-Validation 90-10 split to compare to papers

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
ratings['rating'].value_counts()

4.0    26818
3.0    20047
5.0    13211
3.5    13136
4.5     8551
2.0     7551
2.5     5550
1.0     2811
1.5     1791
0.5     1370
Name: rating, dtype: int64

In [9]:
temptrain,tempvalid=train_test_split(ratings,test_size=0.1,train_size=0.9,random_state=11,shuffle=True)

In [10]:
temptrain['random_dstype']='train'
tempvalid['random_dstype']='test'

/home/kirana/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/kirana/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [11]:
df=pd.concat([temptrain,tempvalid],axis=0)

In [12]:
df.shape

(100836, 10)

## Keeping the last 10% of the ratings for each user as test

In [13]:
df.sort_values(by=['userId','timestamp'],inplace=True)

In [14]:
df['seqNum']=df.groupby('userId').cumcount()+1

In [15]:
df.head()

,userId,movieId,rating,timestamp,title,genres,imdbId,tmdbId,tag,random_dstype,seqNum
73,1,1210,5.0,2000-07-30 18:08:19,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Sci-Fi,86190,1892.0,NaN,train,1
43,1,804,4.0,2000-07-30 18:08:19,She's the One (1996),Comedy|Romance,117628,11363.0,NaN,train,2
171,1,2628,4.0,2000-07-30 18:08:43,Star Wars: Episode I - The Phantom Menace (1999),Action|Adventure|Sci-Fi,120915,1893.0,NaN,train,3
183,1,2826,4.0,2000-07-30 18:08:43,"13th Warrior, The (1999)",Action|Adventure|Fantasy,120657,1911.0,NaN,train,4
120,1,2018,5.0,2000-07-30 18:08:43,Bambi (1942),Animation|Children|Drama,34492,3170.0,NaN,test,5


In [16]:
tempdfmaxseqnum=df.groupby('userId',as_index=False)['seqNum'].max()

In [17]:
tempdfmaxseqnum.head()

,userId,seqNum
0,1,232
1,2,29
2,3,39
3,4,216
4,5,44


In [18]:
tempdfmaxseqnum.rename({'seqNum':'maxSeqNum'},axis=1,inplace=True)

In [19]:
tempdfmaxseqnum.head()

,userId,maxSeqNum
0,1,232
1,2,29
2,3,39
3,4,216
4,5,44


In [20]:
df.shape

(100836, 11)

In [21]:
df=pd.merge(df,tempdfmaxseqnum,on='userId',how='left')
df.shape

(100836, 12)

In [22]:
df.head()

,userId,movieId,rating,timestamp,title,genres,imdbId,tmdbId,tag,random_dstype,seqNum,maxSeqNum
0,1,1210,5.0,2000-07-30 18:08:19,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Sci-Fi,86190,1892.0,NaN,train,1,232
1,1,804,4.0,2000-07-30 18:08:19,She's the One (1996),Comedy|Romance,117628,11363.0,NaN,train,2,232
2,1,2628,4.0,2000-07-30 18:08:43,Star Wars: Episode I - The Phantom Menace (1999),Action|Adventure|Sci-Fi,120915,1893.0,NaN,train,3,232
3,1,2826,4.0,2000-07-30 18:08:43,"13th Warrior, The (1999)",Action|Adventure|Fantasy,120657,1911.0,NaN,train,4,232
4,1,2018,5.0,2000-07-30 18:08:43,Bambi (1942),Animation|Children|Drama,34492,3170.0,NaN,test,5,232


In [23]:
df['time_dstype']=np.where(df['seqNum']<=0.9*df['maxSeqNum'],'train','test')

In [24]:
df.head()

,userId,movieId,rating,timestamp,title,genres,imdbId,tmdbId,tag,random_dstype,seqNum,maxSeqNum,time_dstype
0,1,1210,5.0,2000-07-30 18:08:19,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Sci-Fi,86190,1892.0,NaN,train,1,232,train
1,1,804,4.0,2000-07-30 18:08:19,She's the One (1996),Comedy|Romance,117628,11363.0,NaN,train,2,232,train
2,1,2628,4.0,2000-07-30 18:08:43,Star Wars: Episode I - The Phantom Menace (1999),Action|Adventure|Sci-Fi,120915,1893.0,NaN,train,3,232,train
3,1,2826,4.0,2000-07-30 18:08:43,"13th Warrior, The (1999)",Action|Adventure|Fantasy,120657,1911.0,NaN,train,4,232,train
4,1,2018,5.0,2000-07-30 18:08:43,Bambi (1942),Animation|Children|Drama,34492,3170.0,NaN,test,5,232,train


In [25]:
pd.crosstab(df['random_dstype'],df['time_dstype'])

time_dstype,test,train
random_dstype,,
test,1057,9027
train,9301,81451


## Prepare data

In [26]:
user_to_idx={j:i+1 for i, j in enumerate(df['userId'].unique())}
movie_to_idx={j:i+1 for i,j in enumerate(df['movieId'].unique())}
idx_to_user={i+1:j for i, j in enumerate(df['userId'].unique())}
idx_to_movie={i+1:j for i,j in enumerate(df['userId'].unique())}

In [27]:
df['user_idx']=[user_to_idx[i] for i in df['userId']]
df['movie_idx']=[movie_to_idx[i] for i in df['movieId']]

In [28]:
df['dstype_random_train']=np.where(df['random_dstype']=='train',1,0)
df['dstype_time_train']=np.where(df['time_dstype']=='train',1,0)

In [29]:
df['dstype_random_valid']=np.where(df['random_dstype']!='train',1,0)
df['dstype_time_valid']=np.where(df['time_dstype']!='train',1,0)

In [30]:
df_ratings=df.pivot(index='userId',columns='movieId',values='rating')
df_ratings.fillna(0,inplace=True)

In [31]:
dfflagtrain=df.pivot(index='userId',columns='movieId',values='dstype_random_train')
dfflagtrain.head()
dfflagtrain.fillna(0,inplace=True)
df_train=df_ratings*dfflagtrain

In [34]:
dfflagvalid=df.pivot(index='userId',columns='movieId',values='dstype_random_valid')
dfflagvalid.head()
dfflagvalid.fillna(0,inplace=True)
df_valid=df_ratings*dfflagvalid

In [35]:
df.head()

,userId,movieId,rating,timestamp,title,genres,imdbId,tmdbId,tag,random_dstype,seqNum,maxSeqNum,time_dstype,user_idx,movie_idx,dstype_random_train,dstype_time_train,dstype_random_valid,dstype_time_valid
0,1,1210,5.0,2000-07-30 18:08:19,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Sci-Fi,86190,1892.0,NaN,train,1,232,train,1,1,1,1,0,0
1,1,804,4.0,2000-07-30 18:08:19,She's the One (1996),Comedy|Romance,117628,11363.0,NaN,train,2,232,train,1,2,1,1,0,0
2,1,2628,4.0,2000-07-30 18:08:43,Star Wars: Episode I - The Phantom Menace (1999),Action|Adventure|Sci-Fi,120915,1893.0,NaN,train,3,232,train,1,3,1,1,0,0
3,1,2826,4.0,2000-07-30 18:08:43,"13th Warrior, The (1999)",Action|Adventure|Fantasy,120657,1911.0,NaN,train,4,232,train,1,4,1,1,0,0
4,1,2018,5.0,2000-07-30 18:08:43,Bambi (1942),Animation|Children|Drama,34492,3170.0,NaN,test,5,232,train,1,5,0,1,1,0


In [36]:
genre_to_idx={j:i+1 for i,j in enumerate(df['genres'].unique())}

In [37]:
len(genre_to_idx)

951

In [39]:
df['genre_idx']=[genre_to_idx.get(x) for x in df['genres']]

In [40]:
df['tag'].nunique()

1087

In [43]:
tag_to_idx={j:i+1 for i,j in enumerate(df['tag'].unique())}

In [44]:
len(tag_to_idx)

1088

In [45]:
df['tag_idx']=[tag_to_idx.get(x) for x in df['tag']]

In [46]:
import pickle

In [47]:
pickle.dump([df,df_train,df_valid,df,df_ratings,dfflagtrain,dfflagvalid,idx_to_user,\
             idx_to_movie,movie_to_idx,user_to_idx],open(f'{DATAPATH}/reads.pkl','wb'))

In [48]:
df_ratings=df.pivot(index='movieId',columns='userId',values='rating')
df_ratings.fillna(0,inplace=True)

In [49]:
dfflagtrain=df.pivot(index='movieId',columns='userId',values='dstype_random_train')
dfflagtrain.head()
dfflagtrain.fillna(0,inplace=True)
df_train=df_ratings*dfflagtrain

In [50]:
dfflagvalid=df.pivot(index='movieId',columns='userId',values='dstype_random_valid')
dfflagvalid.head()
dfflagvalid.fillna(0,inplace=True)
df_valid=df_ratings*dfflagvalid

In [51]:
pickle.dump([df,df_train,df_valid,df,df_ratings,dfflagtrain,dfflagvalid,idx_to_user,\
             idx_to_movie,movie_to_idx,user_to_idx],open(f'{DATAPATH}/itemreads.pkl','wb'))